#Langchain Permit Retrievers
This module provides two specialized retrievers that integrate Permit.io's authorization capabilities with Langchain's retrieval systems:

* ReBACSelfQueryRetriever: Implements Relationship-Based Access Control (ReBAC) using natural language queries

* RBACEnsembleRetriever: Combines semantic search with Role-Based and Attribute-Based Access Control (RBAC/ABAC)

## Prerequisites

```python
from permit import Permit
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
# ... other imports as needed

# Initialize Permit client
permit_client = Permit(
    token="your_permit_api_key",
    pdp="your_pdp_url"
)
```

## ReBACSelfQueryRetriever
This retriever extends Langchain's SelfQueryRetriever to include relationship-based access control through Permit.io integration.

### Basic Usage

```python
# Initialize vector store with documents
docs = [
    Document(
        page_content="Confidential project proposal",
        metadata={
            "owner": "user-123",
            "relationships": ["team-a", "managers"],
            "resource_type": "document"
        }
    )
]

# Create retriever
rebac_retriever = ReBACSelfQueryRetriever(
    llm=ChatOpenAI(),
    vectorstore=Chroma.from_documents(docs, OpenAIEmbeddings()),
    permit_client=permit_client
)

# Query with relationship context
docs = await rebac_retriever.aget_relevant_documents(
    "Find project proposals",
    user_context={"user_id": "user-123", "relationships": ["team-a"]}
)
```

### Metadata Schema
The retriever uses the following metadata fields:

* owner: Document owner identifier
* relationships: List of relationship identifiers that have access
* resource_type: Type of the resource for permission checking


## RBACEnsembleRetriever
This retriever combines multiple search strategies with role-based and attribute-based access control.

### Basic Usage

```python
# Initialize with documents
docs = [
    Document(
        page_content="HR Policy Document",
        metadata={
            "department": "HR",
            "classification": "internal",
            "required_role": "hr_staff"
        }
    )
]

# Create retrievers
semantic_retriever = Chroma.from_documents(
    docs, 
    OpenAIEmbeddings()
).as_retriever()

permission_retriever = BM25Retriever.from_documents(docs)

# Create ensemble
rbac_retriever = RBACEnsembleRetriever(
    retrievers=[semantic_retriever, permission_retriever],
    weights=[0.6, 0.4],
    permit_client=permit_client
)

# Query with role context
docs = await rbac_retriever.aget_relevant_documents(
    "HR policies",
    user_context={
        "roles": ["hr_staff"],
        "attributes": {"department": "HR"}
    }
)
```
## Access Control
The retriever supports:

* Role-Based Access: Using the roles field in user context
* Attribute-Based Access: Using the attributes field in user context
* Weighted Results: Combining semantic relevance with permission-based filtering

## Advanced Features
### Custom Permission Logic
Both retrievers support custom permission logic through Permit.io configurations:

```python
# Example of custom permission check
allowed = await permit_client.check(
    user=user_context,
    action="read",
    resource={
        "type": doc.metadata.get("resource_type"),
        "attributes": doc.metadata
    }
)
```